In [27]:
import s3fs
import os
import os.path as op
import pandas as pd
import numpy as np

import afqinsight as afqi

In [28]:
subjects = ["100206", "162228", "175540", "100004", "100307", "100408", "100610", "101006", "101107", "101208"]
subjects_num = [int(i) for i in subjects] 

In [29]:
# download profiles
fs = s3fs.S3FileSystem()

if not op.exists('hcp_data'):
    os.mkdir('hcp_data')

for subject in subjects:
    if not op.exists(f'hcp_data/sub-{subject}'):
        os.mkdir(f'hcp_data/sub-{subject}')
    try:
        fs.get(f'profile-hcp/afq/afq/sub-{subject}/sess-01/sub-{subject}_dwi_profiles.csv', f'hcp_data/sub-{subject}/nodes.csv')
    except FileNotFoundError:
        print(f'subject {subject} profiles not found')

subject 100004 profiles not found
subject 100307 profiles not found
subject 100408 profiles not found
subject 100610 profiles not found
subject 101006 profiles not found
subject 101107 profiles not found
subject 101208 profiles not found


In [4]:
# organize profiles
profile_all = pd.DataFrame(columns=['subjectID', 'tractID', 'nodeID', 'fa' , 'md'])
for subject in subjects:
    profile_curr = pd.read_csv(f'hcp_data/sub-{subject}/nodes.csv')
    for index, row in profile_curr[profile_curr['scalar'] == 'dti_fa'].iterrows():
        md = profile_curr[(profile_curr['scalar'] == 'dti_md') &
                         (profile_curr['bundle'] == row['bundle']) &
                         (profile_curr['node'] == row['node'])]['profiles']
        profile_all = profile_all.append({'subjectID':subject,
                            'tractID':row['bundle'],
                            'nodeID':row['node'],
                            'fa':row['profiles'],
                            'md':md.to_numpy()[0]}, ignore_index=True)
profile_all.to_csv(f'hcp_data/nodes.csv', index=False)
     

In [5]:
# organize subjects
subjects_pd = pd.read_csv('hcp_data/subjects.csv')
columns = subjects_pd.columns.values
if 'subjectID' not in columns:
    columns[0] = 'subjectID'
    subjects_pd.columns = columns
    subjects_pd.to_csv('hcp_data/subjects.csv', index=True)

In [8]:
subjects_pd = pd.read_csv('hcp_data/subjects.csv', index_col="subjectID").drop(
        ["Unnamed: 0"], axis="columns"
    )
afq_data = afqi.load_afq_data('hcp_data', "Endurance_AgeAdj") # subjects_pd.columns

In [20]:

x, y, groups, cols = afq_data.x, afq_data.y.loc[subjects_num].to_numpy(), afq_data.groups, afq_data.columns
print(x.shape)
print(y)
print(x)

(3, 3801)
[121.27 117.31 108.23]
[[2.21025703e-01 3.06240538e-01 4.09478040e-01 ... 6.83155576e-04
  7.15429303e-04 1.00000000e+00]
 [2.01698555e-01 2.90885761e-01 4.40072899e-01 ... 5.98025171e-04
  6.30305592e-04 1.00000000e+00]
 [2.06613454e-01 2.79463243e-01 3.53475105e-01 ... 6.44196943e-04
  6.64731768e-04 1.00000000e+00]]


In [23]:
hp_cv_res_auc = afqi.fit_hyperparams_cv(
    x, y, groups, max_evals_per_cv=1000,
    score='rmse',
    loss_type='square',
    trials_pickle_dir='./cv_trials_cv2_rs42_rmse',
    verbose=1, random_state=42, clf_threshold=0.5,
    n_splits=2
)

job exception: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=1.


ValueError: Cannot have number of splits n_splits=3 greater than the number of samples: n_samples=1.